In [4]:
import numpy as np
import robustsp as rsp
from scipy.optimize import least_squares as lsq

def arma_est_bip_tau(x,p,q):
    # Robust starting point by BIP AR-tau approximation
    beta_initial = rsp.robust_starting_point(x,p,q)[0]
    
    F = lambda beta: rsp.arma_tau_resid_sc(x,beta,p,q)
    
    F_bip = lambda beta: rsp.bip_tau_resid_sc(x,beta,p,q)[0]

    beta_arma = lsq(F, beta_initial,xtol=5*1e-7,ftol=5*1e-7,method='lm')['x']
    
    beta_bip = lsq(F_bip, beta_initial, xtol=5*1e-7,ftol=5*1e-7,method='lm')['x']
    
    a_sc = rsp.arma_tau_resid_sc(x,beta_arma,p,q) # innovations tau-scale for ARMA model
    
    a_bip_sc, x_filt = rsp.bip_tau_resid_sc(x, beta_bip, p, q) # innovations tau-scale for BIP-ARMA model
    
    # final parameter estimate uses the model that provides smallest tau_scale
    beta_hat = beta_arma_mm if a_sc<a_bip_sc else beta_bip
    
    # final tau scale
    a_tau_sc = min(a_sc, a_bip_sc)
    
    # Output the results
    
    results = {'ar_coeffs': -beta_hat[:p],
               'ma_coeffs': -beta_hat[p:],
               'inno_scale': a_tau_sc,
               'cleaned_signal': x_filt,
               'ar_coeffs_init': -1*beta_initial[:p],
               'ma_coeffs_init': -1*beta_initial[p:]}
    
    return results

import scipy.io as io
x = io.loadmat('x_Ao')['x_ao'].flatten()
p=1
q=0
arma_est_bip_tau(x,1,0) # stimmt ungefähr
arma_est_bip_tau(x,0,1)
arma_est_bip_tau(x,2,1) 

ValueError: Method 'lm' doesn't work when the number of residuals is less than the number of variables.